# 스타벅스 매장 데이터와 지도


* json_normalize(): JSON 데이터 DataFrame으로 전환
* folium 을 사용하여 지도에 위치 표시

# 스타벅스 매장찾기
http://www.istarbucks.co.kr

* STORE >> 매장 찾기
* 지역검색 >> 서울 >> 전체

In [1]:
import requests
import json
import pandas as pd
from pandas import *

%config InlineBackend.figure_format = 'retina'

In [2]:
data = {
    'ins_lat':'37.56682', # 지정한 위도와 경도에서 가까운 순으로 나열
    'ins_lng':'126.97865',
    'p_sido_cd':'01', # 01=서울시, 08=경기 ... 16=제주
    'p_gugun_cd':'',  # 세부지역 (지정하지 않으면 시/도 전체)
    'in_biz_cd':'',
    'set_date':'',
    'iend':'1000',
}
    
url = 'https://www.starbucks.co.kr/store/getStore.do'
r = requests.post(url, data=data)

r.text[:1000] # 수신된 데이터의 앞부분만 확인

'{"list":[{"seq":0,"sido_cd":null,"sido_nm":null,"gugun_cd":null,"gugun_nm":null,"code_order":null,"view_yn":null,"store_num":null,"sido":null,"gugun":null,"address":null,"new_img_nm":null,"p_pro_seq":0,"p_view_yn":null,"p_sido_cd":"","p_gugun_cd":"","p_store_nm":null,"p_theme_cd":null,"p_wireless_yn":null,"p_smoking_yn":null,"p_book_yn":null,"p_music_yn":null,"p_terrace_yn":null,"p_table_yn":null,"p_takeout_yn":null,"p_parking_yn":null,"p_dollar_assent":null,"p_card_recharge":null,"p_subway_yn":null,"stb_store_file_renew":null,"stb_store_theme_renew":null,"stb_store_time_renew":null,"stb_store_lsm":null,"s_code":"1509","s_name":"역삼아레나빌딩","tel":"1522-3232","dlvry_call_cntr_phno":null,"fax":"02-568-3763","sido_code":"01","sido_name":"서울","gugun_code":"0101","gugun_name":"강남구","addr":"서울특별시 강남구 역삼동 721-13 아레나빌딩","park_info":null,"new_state":null,"theme_state":"Z9999@T05@T08@T16@T17@T20@T21@T30@T36@@T43@T52@T56@T57@T64@T65@T69@T71@P80@P90","new_bool":0,"search_text":"","ins_lat":"","ins_l

데이터는 JSON으로 수신되며, 형태는 다음과 같다.

```json
{
  "list": [
      {
            // ... 중략 ...
            "s_code": "1311",
            "s_name": "방화DT",
            "tel": "02-2664-3480",
            "fax": "02-2664-3481",
            "sido_code": "01",
            "sido_name": "서울",
            "gugun_code": "0103",
            "gugun_name": "강서구",
            "addr": "서울특별시 강서구 방화동 293-4",
            "park_info": null,
            "new_state": null,
            "theme_state": "T17@T16@T09@T20@T01@T05@T08@T04",
            // ... 중략 ...
            "lat": "37.574339",
            "lot": "126.816415",
            "t22": 0
        },
        {
            // ... 중략 ...
            "s_code": "1267",
            "s_name": "마곡나루역",
            "tel": "02-3662-3504",
            "fax": "02-3662-3505",
            "sido_code": "01",
            "sido_name": "서울",
            "gugun_code": "0103",
            "gugun_name": "강서구",
            "addr": "서울특별시 강서구 마곡동 759-3 보타닉파크타워Ⅰ105,203,204호",
            "park_info": null,
            "new_state": null,
            "theme_state": "T08@T05@T04@T17@T16@P80@T20",
            // ... 중략 ...
            "lat": "37.56813",
            "lot": "126.82614",
            "t22": 0
        },
    ]
 }
```

# JSON to DataFrame

json_normalize() 를 사용하여 JSON 데이터를 DataFrame로 전환 

In [4]:
#r.text

In [5]:
jo = json.loads(r.text)
#print(jo)

In [8]:
df = json_normalize(jo, 'list')  ## 정규화
#print(df)

In [9]:
# 행(row)수, 서울 643개 매장
len(df) 

643

In [10]:
# 컬럼수 137개
df.columns

Index(['seq', 'sido_cd', 'sido_nm', 'gugun_cd', 'gugun_nm', 'code_order',
       'view_yn', 'store_num', 'sido', 'gugun',
       ...
       't30', 't36', 't27', 't29', 't43', 't48', 'z9999', 't64', 't66', 'p02'],
      dtype='object', length=137)

In [11]:
# 주요한 컬럼 몇 가지 선택
df = df[ ['s_name', 'lat', 'lot', 'sido_name', 'gugun_name', 'doro_address', 'tel'] ] 
df.head(4)

,s_name,lat,lot,sido_name,gugun_name,doro_address,tel
0,역삼아레나빌딩,37.501087,127.043069,서울,강남구,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,서울,강남구,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.5139309,127.0206057,서울,강남구,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,서울,강남구,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232


# 위도 경도 데이터 타입 변환

In [12]:
df.dtypes

s_name          object
lat             object
lot             object
sido_name       object
gugun_name      object
doro_address    object
tel             object
dtype: object

### lat, lot타입으로 전환 (str→float) 

In [13]:
df['lat'] = df['lat'].astype(float)
df['lot'] = df['lot'].astype(float)

In [14]:
df.dtypes

s_name           object
lat             float64
lot             float64
sido_name        object
gugun_name       object
doro_address     object
tel              object
dtype: object

### 특정 지점

In [15]:
df[df['s_name'] == '논현역사거리'] # 시청점

,s_name,lat,lot,sido_name,gugun_name,doro_address,tel
1,논현역사거리,37.510178,127.022223,서울,강남구,서울특별시 강남구 강남대로 538 (논현동),1522-3232


스타벅스 시청점의 위도(lat)와 경도(lot)는 각각  37.56629, 126.979808

# folium
https://github.com/python-visualization/folium

파이썬 지리정보 시각화 모듈  (문서: https://folium.readthedocs.io )

!pip install folium

In [17]:
import folium

# 서울 시청 
map_osm = folium.Map(location=(37.56629, 126.979808))
map_osm

지도를 상하좌우로 움직이거나 확대/축소 할 수 있다

In [18]:
# 확대 지정 (zoom_start)
map_osm = folium.Map(location=(37.56629, 126.979808), zoom_start=17)
map_osm

In [21]:
# 지도 tiles 지정
map_osm = folium.Map(location=(37.56629, 126.979808), zoom_start=17, tiles='CartoDB Positron')
# “OpenStreetMap”, “CartoDB Positron”, “CartoDB Voyager”
map_osm

In [22]:
시청_좌표=(37.56629, 126.979808)
 
map_osm = folium.Map(location=시청_좌표, zoom_start=17)
folium.Marker(시청_좌표, popup='시청').add_to(map_osm)
map_osm

In [23]:
map = folium.Map(location=[37.56629, 126.979808], tiles='OpenStreetMap', zoom_start=13)
folium.Marker([37.56629, 126.979808], popup='The Waterfront').add_to(map)
folium.CircleMarker([37.56629, 126.979808], radius=50,
                   popup='시청', color='red',
                   fill_color='#3186cc').add_to(map)
map

# 서울 스타벅스 전 지점

In [24]:
map_osm = folium.Map(location=시청_좌표, zoom_start=11)

for ix, row in df.iterrows():
    location = (row['lat'], row['lot'])
    folium.Marker(location, popup=row['s_name']).add_to(map_osm)

map_osm

# 제주도 스타벅스 매장 지도

In [25]:
import requests
import json
import pandas as pd
한라산_좌표 = (33.361666, 126.529166)
data = {
    'ins_lat': '33.4996213', # 제주 시청의 좌표
    'ins_lng': '126.5311884',
    'p_sido_cd':'16', # 01=서울시, 08=경기 ... 16=제주
    'p_gugun_cd':'',  # 세부지역 (지정하지 않으면 시/도 전체)
    'in_biz_cd':'',
    'set_date':'',
    'iend':'1000',
}
    
url = 'https://www.starbucks.co.kr/store/getStore.do'
r = requests.post(url, data=data)
df = json_normalize(json.loads(r.text), 'list')

map_osm = folium.Map(location=(33.361666, 126.529166), zoom_start=10)
for ix, row in df.iterrows():
    location = (row['lat'], row['lot'])
    folium.Marker(location, popup=row['s_name']).add_to(map_osm)

map_osm

In [26]:
df[['s_name', 'gugun_name', 'doro_address', 'tel']]

,s_name,gugun_name,doro_address,tel
0,제주신화월드R,서귀포시,제주특별자치도 서귀포시 안덕면 신화역사로304번길 38 B1-29,1522-3232
1,제주에듀시티,서귀포시,제주특별자치도 서귀포시 대정읍 에듀시티로 36,1522-3232
2,제주성산DT,서귀포시,제주특별자치도 서귀포시 성산읍 일출로 80,1522-3232
3,제주중문DT,서귀포시,제주특별자치도 서귀포시 천제연로 95(색달동),1522-3232
4,제주송악산,서귀포시,제주특별자치도 서귀포시 대정읍 형제해안로 322,1522-3232
5,서귀포DT,서귀포시,제주특별자치도 서귀포시 일주서로 11 (강정동),1522-3232
6,성산일출봉,서귀포시,"제주특별자치도 서귀포시 성산읍 일출로 284-5 1,2층",1522-3232
7,제주중문,서귀포시,제주특별자치도 서귀포시 중문관광로110번길 32 (색달동),1522-3232
8,서귀포올레,서귀포시,제주특별자치도 서귀포시 중정로 69 (서귀동),1522-3232
9,제주서귀포토평DT,서귀포시,제주특별자치도 서귀포시 일주동로 8516 (토평동),1522-3232


In [30]:
from geopy.geocoders import Nominatim
nom = Nominatim(user_agent = 'South Korea', timeout=None)
n = nom.geocode("London Tower")
print(n.latitude, n.longitude)
print(n.address)

62.9276526 -150.6445964
London Tower, Matanuska-Susitna Borough, Alaska, United States


In [31]:
n = nom.geocode("고려대학교")
print(n.latitude, n.longitude)
print(n.address)

37.5879631 127.0323459
고려대학교 서울캠퍼스, 145, 안암로, 제기동, 용신동, 동대문구, 서울특별시, 02841, 대한민국


In [32]:
map_3 = folium.Map(location=[n.latitude, n.longitude], tiles='openstreetmap', zoom_start=20)
folium.Marker([n.latitude, n.longitude], popup='The Waterfront').add_to(map_3)
folium.CircleMarker([n.latitude, n.longitude], radius=10,
                   popup='My Lab', color='red',
                   fill_color='#3186cc').add_to(map_3)
map_3

# 요약
* requests.post() 데이터 가져오기
* json_normalize(): JSON 데이터 DataFrame으로 전환
* folium 을 사용하여 지도에 위치 표시

### 2018 FinanceData.KR [fb.com/financedata](fb.com/financedata)